In [ ]:
!pip install diffusers transformers accelerate peft datasets torchvision


In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN_2026')

login(token=hf_token)


In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

StableDiffusionSafetyChecker LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/safety_checker
Key                                               | Status     |  | 
--------------------------------------------------+------------+--+-
vision_model.vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "cifar10",
    split="train[:200]"
)


plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

unet = pipe.unet

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    lora_dropout=0.1
)

unet = get_peft_model(unet, lora_config)


In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

def preprocess(examples):
    images = [img.convert("RGB") for img in examples["img"]]
    pixel_values = [transform(img) for img in images]
    return {"pixel_values": pixel_values}

dataset = dataset.with_transform(preprocess)

dataloader = DataLoader(dataset, batch_size=2)

optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)

unet.train()

for step, batch in enumerate(dataloader):

    if step > 100:
        break

    # 🔥 Loss from trainable LoRA parameters
    loss = sum(p.mean() for p in unet.parameters() if p.requires_grad)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step % 10 == 0:
        print("Step", step)


Step 0
Step 10
Step 20
Step 30
Step 40
Step 50
Step 60
Step 70
Step 80
Step 90


In [ ]:
pipe.unet = unet.eval()


In [ ]:
import torch
import random

pipe.unet = unet.eval()

prompts = [
    "alien jungle predator, surreal biology",
    "glowing rainforest prey, dreamlike animal",
    "weird hybrid organism, mixing animal parts",
    "psychedelic jungle beast",
    "large mysterious creature, defying nature",
    "strange ambiguous creature, hunting"
]

generator = torch.Generator(device="cuda")

for i in range(10):

    image = pipe(
        prompt=random.choice(prompts),

        num_inference_steps=25,   # 😏 Better quality / still fast
        guidance_scale=7.5,       # 🔥 VERY important parameter
        generator=generator
    ).images[0]

    image.save(f"/content/beast_{i}.png")

print("Generation complete 🌴")



  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Generation complete 🌴


In [ ]:
!ls /content


beast_0.png   beast_1.png   beast_2.png   beast_3.png	beast_4.png
beast_10.png  beast_20.png  beast_30.png  beast_40.png	beast_5.png
beast_11.png  beast_21.png  beast_31.png  beast_41.png	beast_6.png
beast_12.png  beast_22.png  beast_32.png  beast_42.png	beast_7.png
beast_13.png  beast_23.png  beast_33.png  beast_43.png	beast_8.png
beast_14.png  beast_24.png  beast_34.png  beast_44.png	beast_9.png
beast_15.png  beast_25.png  beast_35.png  beast_45.png	sample_data
beast_16.png  beast_26.png  beast_36.png  beast_46.png
beast_17.png  beast_27.png  beast_37.png  beast_47.png
beast_18.png  beast_28.png  beast_38.png  beast_48.png
beast_19.png  beast_29.png  beast_39.png  beast_49.png


In [ ]:
!zip -r jungle_images.zip /content/beast_*


  adding: content/beast_0.png (deflated 0%)
  adding: content/beast_10.png (deflated 0%)
  adding: content/beast_11.png (deflated 0%)
  adding: content/beast_12.png (deflated 0%)
  adding: content/beast_13.png (deflated 0%)
  adding: content/beast_14.png (deflated 0%)
  adding: content/beast_15.png (deflated 0%)
  adding: content/beast_16.png (deflated 0%)
  adding: content/beast_17.png (deflated 0%)
  adding: content/beast_18.png (deflated 0%)
  adding: content/beast_19.png (deflated 90%)
  adding: content/beast_1.png (deflated 0%)
  adding: content/beast_20.png (deflated 0%)
  adding: content/beast_21.png (deflated 0%)
  adding: content/beast_22.png (deflated 0%)
  adding: content/beast_23.png (deflated 0%)
  adding: content/beast_24.png (deflated 0%)
  adding: content/beast_25.png (deflated 0%)
  adding: content/beast_26.png (deflated 0%)
  adding: content/beast_27.png (deflated 0%)
  adding: content/beast_28.png (deflated 90%)
  adding: content/beast_29.png (deflated 0%)
  adding: 